##**2 Assignment**

###**다음과 같은 CNN 모델을 작성해보자**
* Input
  * Input type: torch.Tensor
  * Input shape: (?, 1, 28, 28)
    * 여러장의, 흑백, 28x28 size의 이미지라고 가정하자
* Layers
  * Layer1
    * Conv2d >> C: 32, Kernel size (필터 크기): 3, Stride: 1, Padding: 1
    * ReLU
    * MaxPool >> Kernel size: 2, Stride: 2
    * 입-출력 (?, 1, 28, 28) >> (?, 32, 14, 14)
  * Layer2
    * Conv2d >> C: 64, Kernel size (필터 크기): 3, Stride: 1, Padding: 1
    * ReLU
    * MaxPool >> Kernel size: 2, Stride: 2
    * 입-출력 (?, 32, 14, 14) >> (?, 64, 7, 7)
  * Layer3
    * Conv2d >> C: 128, Kernel size (필터 크기): 3, Stride: 1, Padding: 1
    * ReLU
    * MaxPool >> Kernel size: 2, Stride: 2, Padding: 1
    * 입-출력 (?, 64, 7, 7) >> (?, 128, 4, 4)
  * Layer4
    * Linear >> input: 4x4x128 output: 625
    * ReLU
    * Dropout
    * 입-출력 (4x4x128) >> (625)
  * Layer5
    * Linear >> input: 625 output: 10
    * Softmax (pytorch의 Cross Entropy Loss 함수를 사용하는 것을 감안한다)

In [3]:
from torch import nn
import torch
import numpy as np
import torch.nn.functional as func
import torch.optim as opt
from torchvision import datasets, transforms
import pandas as pd

batch_size = 64
train_data = datasets.MNIST(root='./data/', train=True,
                               transform=transforms.ToTensor(), 
                               download=True)
test_data = datasets.MNIST(root='./data/', train=False,
                               transform=transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(dataset=train_data,
                                           batch_size=batch_size, 
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_data,
                                          batch_size=batch_size,
                                          shuffle=False)

class Model(nn.Module):
    def __init__(self):
        super(Model,self).__init__()
        #동일한 방법
        '''self.layer1 = nn.Sequential(nn.Conv2d(1, 32, 3, padding=1),
                      nn.ReLU(), nn.MaxPool2d(2, stride=2))
        self.layer2 = nn.Sequential(nn.Conv2d(32, 64, 3, padding=1),
                      nn.ReLU(), nn.MaxPool2d(2, stride=2))
        self.layer3 = nn.Sequential(nn.Conv2d(64, 128, 3, padding=1),
                      nn.ReLU(), nn.MaxPool2d(2, stride=2, padding=1))
        self.layer4 = nn.Sequential(nn.Linear(4*4*128, 625),
                      nn.ReLU(), nn.Dropout())
        self.linear = nn.Linear(625, 10)'''

        #동일한 방법
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1) 
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.mp1 = nn.MaxPool2d(2, stride=2)
        self.mp2 = nn.MaxPool2d(2, stride=2, padding=1)
        self.linear1 = nn.Linear(4*4*128, 625)
        self.linear2 = nn.Linear(625, 10)

    def forward(self,x):
        in_size = x.size(0)
        #동일한 방법
        '''x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = x.reshape(in_size, -1)
        x = self.layer4(x)
        x = self.linear(x) #layer5'''

        #동일한 방법
        x = func.relu(self.mp1(self.conv1(x))) #layer1
        x = func.relu(self.mp1(self.conv2(x))) #layer2
        x = func.relu(self.mp2(self.conv3(x))) #layer3
        x = x.reshape(in_size, -1)
        x = func.dropout(func.relu(self.linear1(x))) #layer4
        x = self.linear2(x) #layer5
        return x

model = Model()
criterion = nn.CrossEntropyLoss()
optimizer = opt.SGD(model.parameters(),lr=0.2, momentum=0.5)

def train(epoch):
    model.train() 
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data) 
        loss = criterion(output, target) 
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0: 
            print('Train Epoch: {} [{}/{} ({:.0f}%)]  Loss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test():
    model.eval() 
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        output = model(data) 
        test_loss = criterion(output, target)
        pred = output.data.max(1, keepdim=True)[1] 
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
        
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset), #accuracy 출력
        100. * correct / len(test_loader.dataset)))
    
for i in range(5):
    train(i)
test()

Train Epoch: 0 [0/60000 (0%)]  Loss: 2.298148
Train Epoch: 0 [6400/60000 (11%)]  Loss: 0.267909
Train Epoch: 0 [12800/60000 (21%)]  Loss: 0.079454
Train Epoch: 0 [19200/60000 (32%)]  Loss: 0.072545
Train Epoch: 0 [25600/60000 (43%)]  Loss: 0.310253
Train Epoch: 0 [32000/60000 (53%)]  Loss: 0.152130
Train Epoch: 0 [38400/60000 (64%)]  Loss: 0.091215
Train Epoch: 0 [44800/60000 (75%)]  Loss: 0.124655
Train Epoch: 0 [51200/60000 (85%)]  Loss: 0.072367
Train Epoch: 0 [57600/60000 (96%)]  Loss: 0.005575
Train Epoch: 1 [0/60000 (0%)]  Loss: 0.059271
Train Epoch: 1 [6400/60000 (11%)]  Loss: 0.040458
Train Epoch: 1 [12800/60000 (21%)]  Loss: 0.018803
Train Epoch: 1 [19200/60000 (32%)]  Loss: 0.009881
Train Epoch: 1 [25600/60000 (43%)]  Loss: 0.139022
Train Epoch: 1 [32000/60000 (53%)]  Loss: 0.117092
Train Epoch: 1 [38400/60000 (64%)]  Loss: 0.095115
Train Epoch: 1 [44800/60000 (75%)]  Loss: 0.009084
Train Epoch: 1 [51200/60000 (85%)]  Loss: 0.043295
Train Epoch: 1 [57600/60000 (96%)]  Loss: 0